# Scrap footy stats and upload to SQL database

In [1]:
# import libraries

import mysql.connector
from mysql.connector import Error
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests

In [2]:
# connect to database

def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host = host_name,
            user = user_name,
            passwd = user_password
        )
        print('MySQL Database connection successful.')
    except Error as err:
        print(f"Error:'{err}'")
    return connection

# Put our MySQL Terminal password

pw = 'Niyuanmike1'

# Database name
db = 'football'
connection = create_server_connection('localhost', 'root', pw)

MySQL Database connection successful.


In [3]:
# function for Connect to Database

def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host = host_name,
            user = user_name,
            passwd = user_password,
            database = db_name
        )
        print('MySQL database connection successfull')
    except Error as err:
        print(f"Error: '{err}'")
    return connection

In [4]:
# function for Execute SQL queries

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print('Query was executed successfully')
    except Error as err:
        print(f"Error: '{err}'")

In [5]:
connection = create_db_connection('localhost', 'root', pw, db)

MySQL database connection successfull


In [41]:
# create player table for raw data, mainly char

drop_exist_table = 'DROP TABLE IF EXISTS player_raw;'

create_player_table = '''
CREATE TABLE player_raw(
name NVARCHAR(40) PRIMARY KEY, 
team NVARCHAR(40) NOT NULL, 
nationality NVARCHAR(3) NOT NULL, 
position NVARCHAR(10), 
age NVARCHAR(10), 
games INT, 
games_starts INT, 
minutes INT, 
minutes_90s FLOAT,
goals INT, 
assists INT, 
goals_pens INT, 
pens_made INT, 
pens_att INT, 
cards_yellow INT, 
cards_red INT, 
goals_per90 FLOAT, 
assists_per90 FLOAT, 
goals_assists_per90 FLOAT, 
goals_pens_per90 FLOAT,  
goals_assists_pens_per90 FLOAT,  
xg FLOAT,  
npxg FLOAT,  
xa FLOAT,  
npxg_xa FLOAT,  
xg_per90 FLOAT,  
xg_xa_per90 FLOAT,  
npxg_per90 FLOAT,  
npxg_xa_per90 FLOAT
);
'''
# connect to the database
connection = create_db_connection('localhost', 'root', pw, db)
execute_query(connection, drop_exist_table)
execute_query(connection, create_player_table)

MySQL database connection successfull
Query was executed successfully
Query was executed successfully


In [42]:
# webscraper

main_url = 'https://fbref.com'
user_agent_headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}
league_url = 'https://fbref.com/en/comps/9/Premier-League-Stats'

def team_scraper(league_url): # get the url link for each PL team
    page = requests.get(league_url, headers = user_agent_headers)
    page_soup = soup(page.content,'html.parser')

    containers = page_soup.find(id='stats_squads_standard_for') 
    teams = containers.find_all('a') # find all teams 
    
    connection = create_db_connection('localhost', 'root', pw, db)

    for idx, team in enumerate(teams):
        team_name = team.text
        team_url = main_url + team['href']

        player_scraper(team_name, team_url)

    print('Scraping Done!\n')

def player_scraper(team_name, team_url):
    page = requests.get(team_url, headers = user_agent_headers)
    page_soup = soup(page.content,'html.parser')

    containers = page_soup.find('tbody')
    players = containers.find_all('tr') # find all players

    for idx, player in enumerate(players):
        player_name = player.th.text.replace('\'','`') # in case name with '\''
        player_url = main_url + player.th.a['href']
        nationality = player.find('td',{'data-stat':'nationality'}).text[-3:]
        position = player.find('td',{'data-stat':'position'}).text.replace(',',' ')
        age = player.find('td',{'data-stat':'age'}).text
        games = player.find('td',{'data-stat':'games'}).text
        games_starts = player.find('td',{'data-stat':'games_starts'}).text
        minutes = player.find('td',{'data-stat':'minutes'}).text
        minutes_90s = player.find('td',{'data-stat':'minutes_90s'}).text
        goals = player.find('td',{'data-stat':'goals'}).text
        assists = player.find('td',{'data-stat':'assists'}).text
        goals_pens = player.find('td',{'data-stat':'goals_pens'}).text
        pens_made = player.find('td',{'data-stat':'pens_made'}).text
        pens_att = player.find('td',{'data-stat':'pens_att'}).text
        cards_yellow = player.find('td',{'data-stat':'cards_yellow'}).text
        cards_red = player.find('td',{'data-stat':'cards_red'}).text
        goals_per90 = player.find('td',{'data-stat':'goals_per90'}).text
        assists_per90 = player.find('td',{'data-stat':'assists_per90'}).text
        goals_assists_per90 = player.find('td',{'data-stat':'goals_assists_per90'}).text
        goals_pens_per90 = player.find('td',{'data-stat':'goals_pens_per90'}).text
        goals_assists_pens_per90 = player.find('td',{'data-stat':'goals_assists_pens_per90'}).text
        xg = player.find('td',{'data-stat':'xg'}).text
        npxg = player.find('td',{'data-stat':'npxg'}).text
        xa = player.find('td',{'data-stat':'xa'}).text
        npxg_xa = player.find('td',{'data-stat':'npxg_xa'}).text
        xg_per90 = player.find('td',{'data-stat':'xg_per90'}).text
        xg_xa_per90 = player.find('td',{'data-stat':'xg_xa_per90'}).text
        npxg_per90 = player.find('td',{'data-stat':'npxg_per90'}).text
        npxg_xa_per90 = player.find('td',{'data-stat':'npxg_xa_per90'}).text

        player_data = '\''+player_name +'\'' + ',' + '\''+ team_name + '\''+ ',' + '\''+ nationality + '\''+ ',' + '\''+ position + '\''+ ',' + '\''+ age + '\''+ ',' + str(int(games or 0)) + ',' + str(int(games_starts or 0)) + ',' + str(int(minutes.replace(',','') or 0)) + ',' +  str(float(minutes_90s or 0)) + ',' +  str(int(goals or 0)) + ',' +  str(int(assists or 0)) + ',' +  str(int(goals_pens or 0)) + ',' +  str(int(pens_made or 0)) + ',' +  str(int(pens_att or 0)) + ',' +  str(int(cards_yellow or 0)) + ',' +  str(int(cards_red or 0)) + ',' +  str(float(goals_per90 or 0)) + ',' +  str(float(assists_per90 or 0)) + ',' +  str(float(goals_assists_per90 or 0)) + ',' +  str(float(goals_pens_per90 or 0)) + ',' +  str(float(goals_assists_pens_per90 or 0)) + ',' +  str(float(xg or 0)) + ',' +  str(float(npxg or 0)) + ',' +  str(float(xa or 0)) + ',' +  str(float(npxg_xa or 0)) + ',' +  str(float(xg_per90 or 0)) + ',' +  str(float(xg_xa_per90 or 0)) + ',' +  str(float(npxg_per90 or 0)) + ',' +  str(float(npxg_xa_per90 or 0))
        insert_data = f"INSERT INTO player_raw VALUES({player_data});"
        
        execute_query(connection, insert_data)

In [43]:
team_scraper(league_url)

MySQL database connection successfull
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Query was executed successfully
Qu